# comercio_ext_auxiliares.tb_referencia_uf_mun
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/UF_MUN_delta`
## 📌 Descrição do arquivo
Referência de **Municípios** com código geográfico e UF.
|Coluna|Descrição|
|---|---|
|`CO_MUN_GEO`|Código geográfico do município|
|`NO_MUN`|Nome do município|
|`NO_MUN_MIN`|Nome (minúsculo/padronizado)|
|`SG_UF`|Sigla da UF|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F, types as T
from delta.tables import DeltaTable
def up(c): return F.upper(F.trim(F.col(c).cast(T.StringType())))

In [0]:
bronzePath  = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/UF_MUN_delta/"
silverTable = "silver_comercio_ext_auxiliares.tb_referencia_uf_mun"

In [0]:
silverSchema = T.StructType([
    T.StructField("CO_MUN_GEO", T.StringType(),   False),
    T.StructField("NO_MUN",     T.StringType(),   False),
    T.StructField("NO_MUN_MIN", T.StringType(),   False),
    T.StructField("SG_UF",      T.StringType(),   False),
    T.StructField("TS_REF",     T.TimestampType(),False),
    T.StructField("NM_ORIGEM",  T.StringType(),   False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze_raw = spark.read.format("delta").load(bronzePath)

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
df_norm = (
    df_bronze
    .withColumn("CO_MUN_GEO", up("CO_MUN_GEO"))
    .withColumn("NO_MUN",     F.col("NO_MUN").cast(T.StringType()))
    .withColumn("NO_MUN_MIN", F.col("NO_MUN_MIN").cast(T.StringType()))
    .withColumn("SG_UF",      up("SG_UF"))
    .withColumn("TS_REF",     F.current_timestamp())
    .withColumn("NM_ORIGEM",  F.lit("/landingbeca2026jan/balancacomercial/uf_mun"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_MUN_GEO").isNotNull())

In [0]:
df_dedup = df_valid.dropDuplicates(["CO_MUN_GEO"])

In [0]:
df_silver = df_dedup.select("CO_MUN_GEO","NO_MUN","NO_MUN_MIN","SG_UF","TS_REF","NM_ORIGEM")

##Carga
> #### **overwrite**

In [0]:
delta_target = DeltaTable.forName(spark, silverTable)
merge_condition = "t.CO_MUN_GEO = s.CO_MUN_GEO"

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "NO_MUN":    "s.NO_MUN",
        "NO_MUN_MIN":"s.NO_MUN_MIN",
        "SG_UF":     "s.SG_UF",
        "TS_REF":    "s.TS_REF",
        "NM_ORIGEM": "s.NM_ORIGEM",
    })
    .whenNotMatchedInsertAll()
    .execute())